# LangChain Tool, Tool Calling, Agent 튜토리얼

이 노트북은 LangChain의 핵심 개념인 **Tool**, **Tool Calling**, **Agent**를 단계별로 학습할 수 있도록 구성되었습니다.

## 📚 학습 목차

1. **Tool 기초** - 첫 번째 Tool 만들기
2. **Tool Calling** - LLM이 Tool을 호출하는 방법
3. **Agent** - 복잡한 문제를 단계별로 해결하기

---

## 🎯 학습 목표

이 노트북을 완료하면:
- Tool을 직접 만들 수 있습니다
- LLM이 Tool을 자동으로 사용하도록 설정할 수 있습니다  
- 복잡한 업무를 처리하는 Agent를 개발할 수 있습니다

---

## ⚙️ 환경 설정

먼저 필요한 패키지를 설치하고 API 키를 설정해야 합니다.


In [1]:
# 필요한 패키지 설치 (주석 해제해서 실행)
# !pip install langchain langchain-openai python-dotenv

# 패키지 import
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from typing import Annotated
import random
from datetime import datetime

# API 키 로드
from dotenv import load_dotenv
load_dotenv()

print("✅ 패키지 로드 완료!")
print("⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!")
print("   (.env 파일에 OPENAI_API_KEY=your_key_here)")


✅ 패키지 로드 완료!
⚠️  OpenAI API 키가 설정되어 있는지 확인하세요!
   (.env 파일에 OPENAI_API_KEY=your_key_here)


## 🎯 Tool Calling 작동 원리

1. **🧠 LLM이 사용자 질문을 분석**
   - "15에 23을 곱한 다음, 47을 빼주세요"
   - → 이건 수학 계산이니까 calculator tool이 필요하겠네!

2. **🔧 LLM이 Tool 호출 계획 수립**
   - 첫 번째: calculator(operation="multiply", a=15, b=23)
   - 두 번째: calculator(operation="subtract", a=결과, b=47)

3. **⚡ Tool 실행**
   - 15 × 23 = 345
   - 345 - 47 = 298

4. **💬 최종 답변 생성**
   - "계산 결과는 298입니다."

---

# 3️⃣ Agent

## Agent란?
**Agent**는 복잡한 문제를 해결하기 위해 여러 Tool을 연속적으로 사용하는 더 똑똑한 시스템입니다.

### Tool Calling vs Agent:
- **Tool Calling**: LLM이 한 번에 Tool을 호출하고 끝
- **Agent**: 문제를 단계별로 나누어 여러 Tool을 순차적으로 사용

### Agent의 특징:
1. 🧠 **생각하기 (Reasoning)**: 문제를 분석하고 계획 수립
2. 🔧 **행동하기 (Action)**: 적절한 Tool 선택하고 실행
3. 👀 **관찰하기 (Observation)**: Tool 결과 확인
4. 🔄 **반복하기**: 목표 달성까지 2-3 과정 반복


In [2]:
# Agent용 Tool들 정의 (쇼핑 도우미 예시)
@tool
def search_product(query: Annotated[str, "검색할 상품명"]) -> list:
    """온라인 쇼핑몰에서 상품을 검색합니다."""
    
    # 가짜 상품 데이터베이스
    products = [
        {"이름": "무선 이어폰", "가격": 89000, "평점": 4.5, "재고": 15},
        {"이름": "블루투스 이어폰", "가격": 65000, "평점": 4.2, "재고": 8},
        {"이름": "노이즈캔슬링 헤드폰", "가격": 159000, "평점": 4.8, "재고": 3},
        {"이름": "게이밍 헤드셋", "가격": 95000, "평점": 4.3, "재고": 12},
    ]
    
    # 검색어와 관련된 상품 찾기
    results = []
    for product in products:
        if query.lower() in product["이름"].lower():
            results.append(product)
    
    print(f"'{query}' 검색 결과: {len(results)}개 상품 발견")
    return results

@tool
def check_inventory(product_name: Annotated[str, "상품명"]) -> dict:
    """특정 상품의 재고를 확인합니다."""
    
    inventory = {
        "무선 이어폰": {"재고": 15, "입고예정": "2024-01-15"},
        "블루투스 이어폰": {"재고": 8, "입고예정": "2024-01-10"},
        "노이즈캔슬링 헤드폰": {"재고": 3, "입고예정": "2024-01-20"},
        "게이밍 헤드셋": {"재고": 12, "입고예정": "2024-01-12"},
    }
    
    result = inventory.get(product_name, {"재고": 0, "입고예정": "미정"})
    print(f"'{product_name}' 재고 확인: {result}")
    return result

@tool
def calculate_shipping_cost(product_price: Annotated[float, "상품 가격"], quantity: Annotated[int, "수량"]) -> dict:
    """배송비를 계산합니다."""
    
    total_price = product_price * quantity
    
    if total_price >= 50000:
        shipping_cost = 0
        shipping_type = "무료배송"
    else:
        shipping_cost = 3000
        shipping_type = "일반배송"
    
    result = {
        "상품금액": total_price,
        "배송비": shipping_cost,
        "총금액": total_price + shipping_cost,
        "배송타입": shipping_type
    }
    
    print(f"배송비 계산 결과: {result}")
    return result

@tool
def add_to_cart(product_name: Annotated[str, "상품명"], quantity: Annotated[int, "수량"]) -> str:
    """상품을 장바구니에 추가합니다."""
    
    cart_id = random.randint(10000, 99999)
    message = f"'{product_name}' {quantity}개가 장바구니(ID: {cart_id})에 추가되었습니다."
    print(message)
    return message

print("🛒 쇼핑 Agent용 Tool들이 준비되었습니다!")
print("- search_product: 상품 검색")
print("- check_inventory: 재고 확인")
print("- calculate_shipping_cost: 배송비 계산")
print("- add_to_cart: 장바구니 추가")


🛒 쇼핑 Agent용 Tool들이 준비되었습니다!
- search_product: 상품 검색
- check_inventory: 재고 확인
- calculate_shipping_cost: 배송비 계산
- add_to_cart: 장바구니 추가


## Agent 생성하기

이제 여러 Tool을 연속적으로 사용하는 똑똑한 Agent를 만들어봅시다!


In [3]:
# 쇼핑 도우미 Agent 생성
def create_shopping_agent():
    """쇼핑 도우미 Agent를 생성합니다."""
    
    # LLM 설정
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        temperature=0,
    )
    
    # Agent용 Tool 목록
    agent_tools = [search_product, check_inventory, calculate_shipping_cost, add_to_cart]
    
    # Agent가 사용할 프롬프트 템플릿
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
당신은 친절한 온라인 쇼핑 도우미 Agent입니다.
고객의 쇼핑을 도와주는 것이 목표입니다.

다음 도구들을 사용할 수 있습니다:
- search_product: 상품 검색
- check_inventory: 재고 확인  
- calculate_shipping_cost: 배송비 계산
- add_to_cart: 장바구니 추가

고객의 요청을 단계별로 처리해주세요:
1. 먼저 상품을 검색합니다
2. 재고를 확인합니다
3. 필요하면 배송비를 계산합니다
4. 고객이 원하면 장바구니에 추가합니다

항상 친절하고 도움이 되는 답변을 해주세요.
        """),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])
    
    # Agent 생성
    agent = create_tool_calling_agent(llm, agent_tools, prompt)
    
    # AgentExecutor 생성 (Agent를 실행하는 엔진)
    agent_executor = AgentExecutor(
        agent=agent,
        tools=agent_tools,
        verbose=True,  # Agent의 사고 과정을 보여줌
        max_iterations=10,  # 최대 반복 횟수
        handle_parsing_errors=True,  # 오류 처리
    )
    
    return agent_executor

# Agent 생성
shopping_agent = create_shopping_agent()
print("🤖 쇼핑 도우미 Agent가 준비되었습니다!")
print("복잡한 쇼핑 요청을 단계별로 처리할 수 있습니다.")


🤖 쇼핑 도우미 Agent가 준비되었습니다!
복잡한 쇼핑 요청을 단계별로 처리할 수 있습니다.


## Agent 실습

복잡한 쇼핑 요청을 Agent가 어떻게 단계별로 처리하는지 확인해봅시다!


In [4]:
# 시나리오 1: 간단한 상품 검색
print("🛒 시나리오 1: 간단한 상품 검색")
print("="*50)

query1 = "이어폰을 찾고 있어요. 어떤 제품들이 있나요?"
print(f"고객 요청: {query1}")
print()

try:
    result1 = shopping_agent.invoke({"input": query1})
    print(f"\n🤖 Agent 최종 답변:")
    print(result1['output'])
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("API 키가 설정되어 있는지 확인해주세요!")


🛒 시나리오 1: 간단한 상품 검색
고객 요청: 이어폰을 찾고 있어요. 어떤 제품들이 있나요?



> Entering new AgentExecutor chain...

Invoking: `search_product` with `{'query': '이어폰'}`


'이어폰' 검색 결과: 2개 상품 발견
[{'이름': '무선 이어폰', '가격': 89000, '평점': 4.5, '재고': 15}, {'이름': '블루투스 이어폰', '가격': 65000, '평점': 4.2, '재고': 8}]다음과 같은 이어폰 제품들이 있습니다:

1. **무선 이어폰**
   - 가격: 89,000원
   - 평점: 4.5
   - 재고: 15개

2. **블루투스 이어폰**
   - 가격: 65,000원
   - 평점: 4.2
   - 재고: 8개

어떤 제품에 관심이 있으신가요? 추가 정보를 원하시거나 장바구니에 추가하고 싶으시면 말씀해 주세요!

> Finished chain.

🤖 Agent 최종 답변:
다음과 같은 이어폰 제품들이 있습니다:

1. **무선 이어폰**
   - 가격: 89,000원
   - 평점: 4.5
   - 재고: 15개

2. **블루투스 이어폰**
   - 가격: 65,000원
   - 평점: 4.2
   - 재고: 8개

어떤 제품에 관심이 있으신가요? 추가 정보를 원하시거나 장바구니에 추가하고 싶으시면 말씀해 주세요!


In [5]:
# 시나리오 2: 복합적인 쇼핑 요청 (여러 Tool 사용)
print("\n" + "="*70)
print("🛒 시나리오 2: 복합적인 쇼핑 요청")
print("="*70)

query2 = "게이밍 헤드셋을 3개 주문하려고 합니다. 재고 확인하고, 배송비 포함 총액 계산해서 장바구니에 넣어주세요."
print(f"고객 요청: {query2}")
print()

try:
    result2 = shopping_agent.invoke({"input": query2})
    print(f"\n🤖 Agent 최종 답변:")
    print(result2['output'])
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("API 키가 설정되어 있는지 확인해주세요!")



🛒 시나리오 2: 복합적인 쇼핑 요청
고객 요청: 게이밍 헤드셋을 3개 주문하려고 합니다. 재고 확인하고, 배송비 포함 총액 계산해서 장바구니에 넣어주세요.



> Entering new AgentExecutor chain...

Invoking: `search_product` with `{'query': '게이밍 헤드셋'}`


'게이밍 헤드셋' 검색 결과: 1개 상품 발견
[{'이름': '게이밍 헤드셋', '가격': 95000, '평점': 4.3, '재고': 12}]
Invoking: `check_inventory` with `{'product_name': '게이밍 헤드셋'}`


'게이밍 헤드셋' 재고 확인: {'재고': 12, '입고예정': '2024-01-12'}
{'재고': 12, '입고예정': '2024-01-12'}
Invoking: `calculate_shipping_cost` with `{'product_price': 95000, 'quantity': 3}`


배송비 계산 결과: {'상품금액': 285000.0, '배송비': 0, '총금액': 285000.0, '배송타입': '무료배송'}
{'상품금액': 285000.0, '배송비': 0, '총금액': 285000.0, '배송타입': '무료배송'}
Invoking: `add_to_cart` with `{'product_name': '게이밍 헤드셋', 'quantity': 3}`


'게이밍 헤드셋' 3개가 장바구니(ID: 64827)에 추가되었습니다.
'게이밍 헤드셋' 3개가 장바구니(ID: 64827)에 추가되었습니다.게이밍 헤드셋 3개의 재고가 충분히 있으며, 총액은 배송비 포함 285,000원입니다. 

이제 '게이밍 헤드셋' 3개가 장바구니에 추가되었습니다. 다른 도움이 필요하시면 말씀해 주세요!

> Finished chain.

🤖 Agent 최종 답변:
게이밍 헤드셋 3개의 재고가 충분히 있으며, 총액은 배송비 포함 285,000원입니다. 

이제 '게이밍 헤드셋' 3개가 장

# 🎓 최종 정리

## 배운 내용 요약

### 1️⃣ **Tool (도구)**
- `@tool` 데코레이터로 함수를 Tool로 변환
- LLM이 사용할 수 있는 기능 단위
- 예: 계산기, 검색기, 데이터 분석기

### 2️⃣ **Tool Calling (도구 호출)**
- LLM이 필요에 따라 Tool을 선택하고 실행
- `bind_tools()`로 LLM에 Tool 연결
- 단발성 작업에 적합

### 3️⃣ **Agent (에이전트)**
- 복잡한 문제를 단계별로 해결
- 여러 Tool을 연속적으로 사용
- **ReAct 패턴**: Reasoning → Action → Observation
- `AgentExecutor`로 실행

## 🔄 Agent의 사고 과정 (ReAct 패턴)

1. **Reasoning (추론)**: "고객이 이어폰을 원하니까 먼저 검색해야겠다"
2. **Action (행동)**: `search_product("이어폰")` 실행
3. **Observation (관찰)**: 검색 결과 확인
4. **Reasoning (추론)**: "재고도 확인해야겠다"
5. **Action (행동)**: `check_inventory("무선 이어폰")` 실행
... 반복 ...

## 🚀 다음 단계
- 더 복잡한 Agent 설계
- 외부 API 연동 Tool 개발  
- 메모리 기능 추가
- 멀티 Agent 시스템 구축

---

**축하합니다! Tool, Tool Calling, Agent의 핵심을 모두 이해했습니다!** 🎉

이제 실무에서 바로 활용할 수 있는 AI Agent를 개발할 수 있습니다!
